In [1]:
# ------------------------------------------
# Big Data Processing
# Assignment 1 - Spark Core & Spark SQL
# Part 3 - Spark SQL
# Michael McAleer R00143621
# ------------------------------------------
import pyspark

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType


def load_dataset_get_data_frame(sp, d_dir):
    """Load the dataset into a data frame with a defined schema.

    :param sp: spark session -- spark session object
    :param d_dir: path to data directory -- str
    :return: processed data frame -- pyspark data frame
    """
    # Define data frame schema
    my_schema = StructType(
        [StructField('status', IntegerType(), True),
         StructField('name', StringType(), True),
         StructField('longitude', FloatType(), True),
         StructField('latitude', FloatType(), True),
         StructField('date_status', StringType(), True),
         StructField('bikes_available', IntegerType(), True),
         StructField('docks_available', IntegerType(), True)])

    # Create the data frame from the source data directory and set schema
    return (sp.read.format('csv').option('delimiter', ';')
            .option('quote', '').option('header', 'false')
            .schema(my_schema).load(d_dir))


###############################################################################

def ex1(df):
    """Exercise 1: Total amount of entries in the dataset.

    :param df: processed input data frame -- pyspark data frame
    """
    # Count the total amount of entries in the df and print
    print('- Total dataset entries: {c}'.format(c=df.count()))


###############################################################################

def ex2(df):
    """Exercise 2: Number of Coca-cola bikes stations in Cork.

    :param df: processed input data frame -- pyspark data frame
    """
    # Extract the distinct station names from input RDD
    distinct = df.select('name').distinct()
    # Output the amount of unique values
    print('- Total bike stations in Cork: {c}'.format(c=distinct.count()))


###############################################################################

def ex3(df):
    """Exercise 3: List of Coca-Cola bike stations.

    Note: It would be more efficient to put this in ex2() and collect from
    there so the distinct value count does not need to be computed twice. In a
    real world scenario this approach would be taken.

    :param df: processed input data frame -- pyspark data frame
    """
    # Extract the distinct station names from input RDD
    distinct_df = df.select('name').distinct()
    # Persist the RDD to memory for the count function
    distinct_df.cache()
    # Output the data
    print('Cork Bike Station List:')
    distinct_df.show(distinct_df.count(), False)


###############################################################################

def ex4(df):
    """Exercise 4: Sort the bike stations by their longitude (East to West).

    :param df: processed input data frame -- pyspark data frame
    """
    # Drop all columns which are not required
    location_df = df.drop('status', 'latitude', 'date_status',
                          'bikes_available', 'docks_available')
    # Drop all duplicates so only unique values remain
    unique_df = location_df.drop_duplicates()
    # Sort the stations by longitude from East to West (descending)
    ordered_df = unique_df.orderBy(unique_df['longitude'].desc())
    # Collect the results and output
    result = ordered_df.collect()
    print('Cork Bike Stations (East -> West):')
    for item in result:
        print(item)


###############################################################################

def ex5(df):
    """Exercise 5: Average number of bikes available at Kent Station.

    :param df: processed input data frame -- pyspark data frame
    """
    # Drop all columns which are not required
    location_df = df.drop('status', 'latitude', 'longitude',
                          'date_status', 'docks_available')
    # Filter the data elements for 'Kent Station', aggregate the bike
    # available count, output the average
    result = location_df.filter(location_df['name'] == 'Kent Station').agg(
        {'bikes_available': 'avg'})
    # Output the result
    print('Kent St. Average Bikes Available:', result.collect()[0][0])


###############################################################################

if __name__ == '__main__':
    # Set the location of the dataset directory
    FILE_STORE = '/FileStore/tables/cork_bike_data/'
    # Create the Spark session
    spark = pyspark.sql.SparkSession.builder.getOrCreate()
    # Set log level
    spark.sparkContext.setLogLevel('WARN')
    # Load dataset and create data frame
    input_df = load_dataset_get_data_frame(spark, FILE_STORE)
    # Call the functions
    for i, ex in enumerate([ex1, ex2, ex3, ex4, ex5]):
        print('\n#-------------#\n'
              '| Exercise: {e} |\n'
              '#-------------#'.format(e=i + 1))
        ex(input_df)


#-------------#
 Exercise: 1 |
#-------------#
- Total dataset entries: 1339200

#-------------#
 Exercise: 2 |
#-------------#
- Total bike stations in Cork: 31

#-------------#
 Exercise: 3 |
#-------------#
Cork Bike Station List:
+----------------------+
name |
+----------------------+
Bandfield |
Peace Park |
South Mall |
Fitzgerald's Park |
St. Fin Barre's Bridge|
South Main St. |
Emmet Place |
Cork City Hall |
College of Commerce |
Lapp's Quay |
Lower Glanmire Rd. |
Cork School of Music |
Bus Station |
Brian Boru Bridge |
Kent Station |
Wandesford Quay |
Camden Quay |
Grattan St. |
South Gate Bridge |
North Main St. |
Mercy Hospital |
St. Patricks St. |
Bishop St. |
Grand Parade |
Gaol Walk |
Clontarf Street |
Coburg St. |
Pope's Quay |
Corn Market St. |
Dyke Parade |
Father Mathew Statue |
+----------------------+


#-------------#
 Exercise: 4 |
#-------------#
Cork Bike Stations (East -> West):
Row(name='Kent Station', longitude=-8.45821475982666)
Row(name='Lower Glanmire Rd.', longitude=-8.464118003845215)
Row(name='Brian Boru Bridge', longitude=-8.465152740478516)
Row(name='Clontarf Street', longitude=-8.465629577636719)
Row(name="Lapp's Quay", longitude=-8.46573543548584)
Row(name='Cork City Hall', longitude=-8.465999603271484)
Row(name='Bus Station', longitude=-8.466950416564941)
Row(name='Cork School of Music', longitude=-8.468092918395996)
Row(name='College of Commerce', longitude=-8.469797134399414)
Row(name='South Mall', longitude=-8.469900131225586)
Row(name='Coburg St.', longitude=-8.47056770324707)
Row(name='Father Mathew Statue', longitude=-8.470627784729004)
Row(name='St. Patricks St.', longitude=-8.472615242004395)
Row(name='Emmet Place', longitude=-8.472704887390137)
Row(name='Camden Quay', longitude=-8.473341941833496)
Row(name='Peace Park', longitude=-8.473475456237793)
Row(name='Grand Parade', longitude=-8.475369453430176)
Row(name='South Gate Bridge', longitude=-8.475865364074707)
Row(name='South Main St.', longitude=-8.476895332336426)
Row(name='Corn Market St.', longitude=-8.47700023651123)
Row(name="Pope's Quay", longitude=-8.477384567260742)
Row(name='North Main St.', longitude=-8.478440284729004)
Row(name='Bishop St.', longitude=-8.479026794433594)
Row(name='Grattan St.', longitude=-8.479779243469238)
Row(name='Wandesford Quay', longitude=-8.480039596557617)
Row(name="St. Fin Barre's Bridge", longitude=-8.481961250305176)
Row(name='Mercy Hospital', longitude=-8.482256889343262)
Row(name='Dyke Parade', longitude=-8.48458480834961)
Row(name='Bandfield', longitude=-8.489136695861816)
Row(name="Fitzgerald's Park", longitude=-8.493412971496582)
Row(name='Gaol Walk', longitude=-8.494174003601074)

#-------------#
 Exercise: 5 |
#-------------#
Kent St. Average Bikes Available: 12.810347222222223